In [ ]:
# ------------------------------------------------------------
# PASO 1 — CONFIGURACIÓN DEL ENTORNO Y VERIFICACIÓN INICIAL
# ------------------------------------------------------------

import arcpy
import os
import sys
from collections import Counter

# 📂 Ruta de trabajo principal
work_folder = r"C:\........" # Ajuste a su ruta y carpeta de trabajo

# 🗺️ Shapefile base con los estratos (campo Name)
lulc_shapefile = os.path.join(work_folder, "LULC.shp") # Ajuste al nombre de su archivo shape
name_field = "Name"  # campo que contiene los estratos

# 🔧 Configuración general de ArcPy
arcpy.env.workspace = work_folder
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

# Intentar activar extensión Spatial (no crítica en este paso)
try:
    arcpy.CheckOutExtension("Spatial")
except:
    print("ℹ️ Nota: No se pudo activar la extensión 'Spatial' (no es crítica para este paso).")

# ✅ Verificar existencia del shapefile
if not arcpy.Exists(lulc_shapefile):
    print(f"❌ ERROR: No se encontró el shapefile en la ruta:\n   {lulc_shapefile}")
    sys.exit(1)

print(f"\n✔️ Shapefile detectado correctamente: {lulc_shapefile}")

# 📋 Listar campos disponibles
fields = arcpy.ListFields(lulc_shapefile)
field_names = [f.name for f in fields]

print("\n📑 Campos disponibles en el shapefile:")
for fn in field_names:
    print(f" - {fn}")

# Verificar existencia del campo 'Name'
if name_field not in field_names:
    print(f"\n⚠️ El campo '{name_field}' no existe. Se creará un nuevo campo con ese nombre.")
    arcpy.AddField_management(lulc_shapefile, name_field, "TEXT", field_length=254)
    print(f"✅ Campo '{name_field}' creado correctamente.")

# 📊 Analizar valores únicos del campo Name
print("\n📊 Resumen de valores únicos del campo 'Name':")
values = []
with arcpy.da.SearchCursor(lulc_shapefile, [name_field]) as cursor:
    for r in cursor:
        if r[0] is not None and str(r[0]).strip() != "":
            values.append(str(r[0]).strip())

counter = Counter(values)
total_features = sum(counter.values())

print(f"\nTotal de entidades con valor en '{name_field}': {total_features}\n")
print("Valores únicos encontrados:")
for i, (val, cnt) in enumerate(counter.items()):
    print(f" - '{val}': {cnt}")

print("\n🎯 Paso 1 completado exitosamente")

In [ ]:
# ------------------------------------------------------------
# PASO 2 — CREACIÓN DE RED (FISHNET) Y PUNTOS AUTOMÁTICAMENTE
# ------------------------------------------------------------

import arcpy
import os

# 📂 Ruta de trabajo
work_folder = r"C:\........"

# 📁 Shapefile base (para obtener la extensión)
lulc_shapefile = os.path.join(work_folder, "LULC.shp")

# 📁 Rutas de salida
fishnet_out = os.path.join(work_folder, "Grid_500m.shp")     # Red (polígonos)
points_out = os.path.join(work_folder, "Points_500m.shp")    # Puntos (centroides)

# ⚙️ Configuración del entorno
arcpy.env.workspace = work_folder
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

# 📐 Sistema de referencia del shapefile base
spatial_ref = arcpy.Describe(lulc_shapefile).spatialReference

# 🔍 Obtener extensión del shapefile base
desc = arcpy.Describe(lulc_shapefile)
extent = desc.extent

# Coordenadas automáticas
origin_coord = f"{extent.XMin} {extent.YMin}"  # esquina inferior izquierda
y_axis_coord = f"{extent.XMin} {extent.YMin + 10}"  # define orientación del eje Y (10 m arriba)
corner_coord = f"{extent.XMax} {extent.YMax}"  # esquina superior derecha

# 🧩 Parámetros de la red
cell_width = 500   # metros
cell_height = 500  # metros

print("\n📦 Creando red de 500 m basada en la extensión de LULC.shp...")

# Crear red (fishnet) y puntos (labels)
arcpy.management.CreateFishnet(
    out_feature_class=fishnet_out,
    origin_coord=origin_coord,
    y_axis_coord=y_axis_coord,
    cell_width=cell_width,
    cell_height=cell_height,
    number_rows=None,
    number_columns=None,
    corner_coord=corner_coord,
    labels="LABELS",
    template=None,
    geometry_type="POLYGON"
)

# La herramienta crea automáticamente un shapefile de puntos (labels)
auto_label = fishnet_out.replace(".shp", "_label.shp")

# Renombrar los puntos
if arcpy.Exists(auto_label):
    arcpy.management.Rename(auto_label, points_out)
    print(f"✅ Red creada: {fishnet_out}")
    print(f"✅ Puntos creados: {points_out}")
else:
    print(f"⚠️ No se encontró el archivo de etiquetas ({auto_label})")

# 📏 Asignar proyección
arcpy.DefineProjection_management(fishnet_out, spatial_ref)
arcpy.DefineProjection_management(points_out, spatial_ref)

print("\n🎯 Paso 2 completado: red y puntos generados automáticamente.")

In [ ]:
# ------------------------------------------------------------
# PASO 3 — RECORTE DE PUNTOS CON ARCPY CLIP
# ------------------------------------------------------------

import arcpy
import os

# 🗂️ Definir rutas (necesarias en cada paso)
work_folder = r"C:\........"
lulc_shapefile = os.path.join(work_folder, "LULC.shp")
points_input = os.path.join(work_folder, "Points_500m.shp")
points_clip = os.path.join(work_folder, "Points_500m_Clip.shp")

# ⚙️ Configuración
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

print("\n🎯 Iniciando Paso 3: recortando puntos con Clip con base en LULC.shp...")

# 🔹 Recortar puntos con base en LULC
arcpy.analysis.Clip(
    in_features=points_input,
    clip_features=lulc_shapefile,
    out_feature_class=points_clip
)

print(f"✅ Puntos recortados guardados como: {points_clip}")
print("\n🎉 Paso 3 completado: los puntos fueron recortados con precisión según el estrato o cobertura")

In [ ]:
# ------------------------------------------------------------
# PASO 4 — CALCULAR COORDENADAS X Y PARA CADA PUNTO
# ------------------------------------------------------------

import arcpy
import os

# 🗂️ Rutas
work_folder = r"C:\........"
points_clip = os.path.join(work_folder, "Points_500m_Clip.shp")

# ⚙️ Configuración
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

print("\n📍 Iniciando Paso 4: calculando coordenadas X e Y para cada punto...")

# 🔹 Verificar y crear campos X y Y si no existen
fields = [f.name for f in arcpy.ListFields(points_clip)]
if "X" not in fields:
    arcpy.AddField_management(points_clip, "X", "DOUBLE")
if "Y" not in fields:
    arcpy.AddField_management(points_clip, "Y", "DOUBLE")

# 🔹 Calcular coordenadas
arcpy.CalculateField_management(
    in_table=points_clip,
    field="X",
    expression="!SHAPE.CENTROID.X!",
    expression_type="PYTHON3"
)

arcpy.CalculateField_management(
    in_table=points_clip,
    field="Y",
    expression="!SHAPE.CENTROID.Y!",
    expression_type="PYTHON3"
)

print(f"✅ Coordenadas X e Y calculadas y guardadas en: {points_clip}")
print("\n🎉 Paso 4 completado: todos los puntos tienen sus coordenadas X e Y")

In [ ]:
# ------------------------------------------------------------
# PASO 5 — CREAR CAMPOS Point_X Y Point_Y CON AJUSTE
# ------------------------------------------------------------

import arcpy
import os

# 🗂️ Ruta del shapefile
work_folder = r"C:\........"
points_clip = os.path.join(work_folder, "Points_500m_Clip.shp")

# ⚙️ Configuración
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

print("\n📍 Iniciando Paso 5: creando campos Point_X y Point_Y con ajustes...")

# 🔹 Verificar y crear campos Point_X y Point_Y si no existen
fields = [f.name for f in arcpy.ListFields(points_clip)]
if "Point_X" not in fields:
    arcpy.AddField_management(points_clip, "Point_X", "DOUBLE")
if "Point_Y" not in fields:
    arcpy.AddField_management(points_clip, "Point_Y", "DOUBLE")

# 🔹 Calcular valores ajustados
arcpy.CalculateField_management(
    in_table=points_clip,
    field="Point_X",
    expression="!X! + 30",
    expression_type="PYTHON3"
)

arcpy.CalculateField_management(
    in_table=points_clip,
    field="Point_Y",
    expression="!Y! + 190",
    expression_type="PYTHON3"
)

print(f"✅ Campos Point_X y Point_Y creados y calculados en: {points_clip}")
print("\n🎉 Paso 5 completado: coordenadas ajustadas listas")

In [ ]:
# ------------------------------------------------------------
# PASO 6 — CREAR LÍNEAS A PARTIR DE PUNTOS (XY TO LINE)
# ------------------------------------------------------------

import arcpy
import os

# 🗂️ Ruta de trabajo
work_folder = r"C:\........"
points_clip = os.path.join(work_folder, "Points_500m_Clip.shp")
points_line = os.path.join(work_folder, "Points_500m_Line.shp")

# ⚙️ Configuración
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

print("\n📍 Iniciando Paso 6: creando líneas desde los puntos ajustados...")

# 🔹 Definir referencia espacial
spatial_ref = arcpy.SpatialReference(32616)  # WGS_1984_UTM_Zone_16N

# 🔹 Crear líneas a partir de los puntos
arcpy.management.XYToLine(
    in_table=points_clip,
    out_featureclass=points_line,
    startx_field="X",
    starty_field="Y",
    endx_field="Point_X",
    endy_field="Point_Y",
    line_type="GEODESIC",
    id_field=None,
    spatial_reference=spatial_ref,
    attributes="NO_ATTRIBUTES"
)

print(f"✅ Shape de líneas creado correctamente: {points_line}")
print("\n🎉 Paso 6 completado: líneas generadas a partir de los puntos ajustados")

In [ ]:
# ------------------------------------------------------------
# PASO 7 — GEOMETRÍA MÍNIMA DE DELIMITACIÓN (ENVELOPE)
# ------------------------------------------------------------

import arcpy
import os

# 🗂️ Ruta de trabajo
work_folder = r"C:\........"
points_line = os.path.join(work_folder, "Points_500m_Line.shp")
points_minbo = os.path.join(work_folder, "Points_500m_MinBo.shp")

# ⚙️ Configuración
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

print("\n📍 Iniciando Paso 7: generando geometría mínima de delimitación...")

# 🔹 Generar geometría mínima (Envelope) agrupando por Point_X y Point_Y
arcpy.management.MinimumBoundingGeometry(
    in_features=points_line,
    out_feature_class=points_minbo,
    geometry_type="ENVELOPE",         # Tipo de geometría
    group_option="LIST",               # Opción de grupo
    group_field="Point_X;Point_Y",    # Campos por los cuales agrupar
    mbg_fields_option="NO_MBG_FIELDS" # No agregar campos adicionales
)

print(f"✅ Shape de geometría mínima creado correctamente: {points_minbo}")
print("\n🎉 Paso 7 completado: geometría mínima de delimitación generada")

In [ ]:
# ------------------------------------------------------------
# PASO 8 — RECORTE DE MÍNIMOS CON LULC Y FILTRADO POR ÁREA ≥ 5000 m²
# ------------------------------------------------------------

import arcpy
import os

# 🗂️ Ruta de trabajo
work_folder = r"C:\........"
minbo_shp = os.path.join(work_folder, "Points_500m_MinBo.shp")
lulc_shp = os.path.join(work_folder, "LULC.shp")
um_shp = os.path.join(work_folder, "UM_500m.shp")
um_filter_shp = os.path.join(work_folder, "UM_500m_filter.shp")

# ⚙️ Configuración
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

print("\n📍 Iniciando Paso 8: recortando geometrías mínimas con LULC...")

# 🔹 Recortar geometrías mínimas con el LULC
arcpy.analysis.Clip(
    in_features=minbo_shp,
    clip_features=lulc_shp,
    out_feature_class=um_shp
)

print(f"✅ Shape recortado generado: {um_shp}")

# 🔹 Agregar campo de área en m2
if "Area_m2" not in [f.name for f in arcpy.ListFields(um_shp)]:
    arcpy.AddField_management(um_shp, "Area_m2", "DOUBLE")

# 🔹 Calcular área en metros cuadrados
arcpy.CalculateField_management(
    in_table=um_shp,
    field="Area_m2",
    expression="!shape.area@SQUAREMETERS!",
    expression_type="PYTHON3"
)

print(f"✅ Área en m² calculada para {um_shp}")

# 🔹 Seleccionar polígonos con área ≥ 5000 m²
arcpy.management.MakeFeatureLayer(um_shp, "um_layer")
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="um_layer",
    selection_type="NEW_SELECTION",
    where_clause="Area_m2 >= 5000"
)

# 🔹 Guardar los polígonos seleccionados
arcpy.management.CopyFeatures("um_layer", um_filter_shp)

print(f"✅ Polígonos filtrados guardados en: {um_filter_shp}")
print("\n🎉 Paso 8 completado: UM recortadas y filtradas por área ≥ 5000 m²")

In [ ]:
# ------------------------------------------------------------
# PASO 9 — ASIGNAR CÓDIGOS ÚNICOS A CADA UM
# ------------------------------------------------------------

import arcpy
import os

# 📁 Carpeta de trabajo
work_folder = r"C:\........"

# 📂 Shape de Unidades Muestrales filtradas
um_shp = os.path.join(work_folder, "UM_500m_filter.shp")

# ✅ Verificar si campo UM_Code ya existe, si no crearlo
fields = [f.name for f in arcpy.ListFields(um_shp)]
if "UM_Code" not in fields:
    arcpy.AddField_management(um_shp, "UM_Code", "TEXT", field_length=10)

# 🔄 Asignar códigos secuenciales UM01, UM02, UM03...
with arcpy.da.UpdateCursor(um_shp, ["UM_Code"]) as cursor:
    counter = 1
    for row in cursor:
        row[0] = f"UM{counter:02d}"  # UM01, UM02, etc.
        cursor.updateRow(row)
        counter += 1

print("\n✅ Se han asignado códigos únicos a todas las UMs en UM_500m_filter.shp")

In [ ]:
# ------------------------------------------------------------
# PASO 10 — INTERSECCIÓN UM ↔ LULC para determinar estrato dominante (ajustado)
# ------------------------------------------------------------

import arcpy
import os

# 📁 Ruta base
work_folder = r"C:\........"
gdb_path = r"C:\.........gdb"

# 📁 Archivos de entrada
um_fc = os.path.join(work_folder, "UM_500m_filter.shp")
lulc_fc = os.path.join(work_folder, "LULC.shp")

# 📁 Archivo intermedio (intersección) → guardado en la GDB
intersect_fc = os.path.join(gdb_path, "um_lulc_intersect")

# 📁 Archivo de salida final
out_fc = os.path.join(work_folder, "UM_500m_Final.shp")

print("\n🎯 Iniciando Paso 10: Intersección UM ↔ LULC para determinar estrato dominante...")

# 1️⃣ Intersecar UM con LULC
arcpy.analysis.Intersect(
    in_features=[[um_fc, ""], [lulc_fc, ""]],
    out_feature_class=intersect_fc
)
print("✅ Intersección completada y guardada en la GDB del proyecto.")

# 2️⃣ Calcular el área de cada parte de intersección (m²)
if "Area_m2" not in [f.name for f in arcpy.ListFields(intersect_fc)]:
    arcpy.management.AddField(intersect_fc, "Area_m2", "DOUBLE")

arcpy.management.CalculateGeometryAttributes(
    intersect_fc,
    [["Area_m2", "AREA"]],
    area_unit="SQUARE_METERS"
)
print("📏 Áreas de intersección calculadas correctamente.")

# 3️⃣ Determinar estrato dominante (mayor área) por UM
print("🧮 Determinando el estrato dominante por UM...")

# Crear diccionario {UM_Code: (estrato, área_max)}
dominant_dict = {}

fields = [f.name for f in arcpy.ListFields(lulc_fc)]
lulc_field = "Stratum" if "Stratum" in fields else fields[-1]

with arcpy.da.SearchCursor(intersect_fc, ["UM_Code", lulc_field, "Area_m2"]) as cursor:
    for row in cursor:
        um_code, name, area = row
        if um_code not in dominant_dict or area > dominant_dict[um_code][1]:
            dominant_dict[um_code] = (name, area)

# 4️⃣ Agregar campo para el estrato dominante (nombre corto <= 10 caracteres)
new_field = "Strat_Dom"
existing_fields = [f.name for f in arcpy.ListFields(um_fc)]
if new_field not in existing_fields:
    arcpy.management.AddField(um_fc, new_field, "TEXT", field_length=50)

# 5️⃣ Actualizar con los valores del estrato dominante
with arcpy.da.UpdateCursor(um_fc, ["UM_Code", new_field]) as cursor:
    for row in cursor:
        um_code = row[0]
        if um_code in dominant_dict:
            row[1] = dominant_dict[um_code][0]
            cursor.updateRow(row)

# 6️⃣ Guardar el shapefile final
arcpy.management.CopyFeatures(um_fc, out_fc)

print(f"\n✅ Paso 10 completado exitosamente.")
print(f"📁 Archivo final guardado en:\n   {out_fc}")
print(f"📍 Capa intermedia (intersección) guardada en la GDB:\n   {intersect_fc}")

In [ ]:
# ------------------------------------------------------------
# PASO 11 — SELECCIÓN ALEATORIA ESTRATIFICADA DE UNIDADES MUESTRALES (40)
# ------------------------------------------------------------
import arcpy
import random
import os

# -----------------------------
# PARÁMETROS (ajusta si hace falta)
# -----------------------------
workspace = r"C:\........"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

# Archivo de unidades muestrales resultante del Paso 10 (debe contener UM_Code y Dominant_LULC)
um_fc = os.path.join(workspace, "UM_500m_Final.shp")    # <- ajusta si tiene otro nombre

# Salida
out_shp = os.path.join(workspace, "DS_Final.shp")

# Tamaño objetivo por estrato (puedes cambiarlo)
target_per_stratum = 40

# Nombre del campo que contiene el código de la UM (texto)
um_code_field = "UM_Code"           # ej. UM01, UM02
# Nombre del campo que contiene el estrato dominante (texto)
dominant_field = "Strat_Dom"

# -----------------------------
# VALIDACIONES iniciales
# -----------------------------
if not arcpy.Exists(um_fc):
    # ¡CORRECCIÓN DE IDENTACIÓN AQUÍ!
    raise SystemExit(f"ERROR: No existe el feature class: {um_fc}")

# Revisar que los campos existan
fields_present = [f.name for f in arcpy.ListFields(um_fc)]
for fld in (um_code_field, dominant_field):
    if fld not in fields_present:
        raise SystemExit(f"ERROR: El campo '{fld}' no existe en {um_fc}. Campos disponibles: {fields_present}")

# -----------------------------
# Crear layer para trabajar
# -----------------------------
um_layer = "um_layer_temp"
if arcpy.Exists(um_layer):
    arcpy.Delete_management(um_layer)
arcpy.MakeFeatureLayer_management(um_fc, um_layer)

# -----------------------------
# Obtener lista de estratos y conteos por estrato
# -----------------------------
strata_counts = {}
strata_to_ids = {}

with arcpy.da.SearchCursor(um_fc, [um_code_field, dominant_field]) as cursor:
    for um_code, dom in cursor:
        if dom is None:
            continue
        dom = str(dom).strip()
        if dom == "":
            continue
        strata_counts.setdefault(dom, 0)
        strata_counts[dom] += 1
        strata_to_ids.setdefault(dom, []).append(str(um_code))

print("\n🔍 Estratos y conteos encontrados:")
for s, c in sorted(strata_counts.items(), key=lambda x: -x[1]):
    print(f"  - {s}: {c} UM disponibles")

# -----------------------------
# Muestreo aleatorio estratificado
# -----------------------------
selected_ids = set()  # usar set para evitar duplicados si un UM aparece en >1 estrato
for strata, ids in strata_to_ids.items():
    n_available = len(ids)
    n_target = min(target_per_stratum, n_available)

    # Ahora se está trabajando fijo por estrato; si se desea proporcional a la superficie en vez de fijo, habría que calcular proporciones;
    chosen = random.sample(ids, n_target) if n_available > n_target else ids.copy()
    for i in chosen:
        selected_ids.add(i)

print(f"\n🎯 Total UM seleccionadas (únicas): {len(selected_ids)}")

if len(selected_ids) == 0:
    raise SystemExit("ERROR: No se seleccionaron unidades muestrales. Revisa los campos y datos.")

# -----------------------------
# Construir cláusula SQL segura (UM_Code es texto)
# -----------------------------
# Usamos comillas simples para SQL: 'UM01','UM02',...
quoted_ids = [f"'{id_}'" for id_ in selected_ids]
where_in = f"{um_code_field} IN ({', '.join(quoted_ids)})"
print(f"\n🔎 Cláusula SQL usada: {where_in}")

# -----------------------------
# Ejecutar selección y exportar
# -----------------------------
arcpy.management.SelectLayerByAttribute(um_layer, "NEW_SELECTION", where_in)
# Verificar cuántos seleccionados realmente en la capa
selected_count = int(arcpy.GetCount_management(um_layer).getOutput(0))
print(f"🔢 Registros seleccionados en la capa: {selected_count}")
if selected_count == 0:
    raise SystemExit("ERROR: La selección produjo 0 registros. Revisa la cláusula SQL o el tipo del campo UM_Code.")

# Guardar como shapefile final
if arcpy.Exists(out_shp):
    arcpy.Delete_management(out_shp)
arcpy.management.CopyFeatures(um_layer, out_shp)
print(f"\n✅ Muestreo final guardado en: {out_shp}")

# Limpieza
arcpy.Delete_management(um_layer)
print("\n🎉 Paso 11 completado: muestreo estratificado creado correctamente")

In [ ]:
# ------------------------------------------------------------
# PASO 12 — RESUMEN DE UNIDADES MUESTRALES POR ESTRATO
# ------------------------------------------------------------

import arcpy
import os
from collections import Counter

# -----------------------------
# PARÁMETROS
# -----------------------------
workspace = r"C:\........"
arcpy.env.workspace = workspace

# Shapefile resultante del muestreo estratificado
ds_final_fc = os.path.join(workspace, "DS_Final.shp")

# Campo de estrato
dominant_field = "Strat_Dom"

# -----------------------------
# VALIDACIÓN
# -----------------------------
if not arcpy.Exists(ds_final_fc):
    raise SystemExit(f"ERROR: No existe el feature class: {ds_final_fc}")

# -----------------------------
# Contar número de UM por estrato
# -----------------------------
strata_counts = Counter()

with arcpy.da.SearchCursor(ds_final_fc, [dominant_field]) as cursor:
    for row in cursor:
        val = row[0]
        if val is not None:
            val = str(val).strip()
            if val != "":
                strata_counts[val] += 1

# -----------------------------
# Mostrar tabla resumen
# -----------------------------
print("\n📊 RESUMEN DE UNIDADES MUESTRALES POR ESTRATO")
print("Estrato".ljust(30), "UM seleccionadas")
print("-"*45)
for s, count in sorted(strata_counts.items(), key=lambda x: -x[1]):
    print(f"{s.ljust(30)} {count}")

total_um = sum(strata_counts.values())
print("-"*45)
print(f"{'TOTAL'.ljust(30)} {total_um}")

print("\n🎉 Paso 12 completado: resumen generado correctamente")

In [ ]:
# ------------------------------------------------------------
# PASO 13 — CREAR PUNTOS CENTRALES DE LAS UM CON COORDENADAS X Y
# ------------------------------------------------------------

import arcpy
import os

# -----------------------------
# PARÁMETROS
# -----------------------------
workspace = r"C:\........"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

# Shapefile de UM final
um_fc = os.path.join(workspace, "DS_Final.shp")

# Salida: puntos centrales
central_points_fc = os.path.join(workspace, "Central_Points.shp")

# Campos a conservar
fields_to_copy = ["UM_Code", "Strat_Dom"]

# -----------------------------
# Crear shapefile de puntos
# -----------------------------
# Obtener la referencia espacial del shapefile de UM
spatial_ref = arcpy.Describe(um_fc).spatialReference

# Crear el feature class de puntos
arcpy.management.CreateFeatureclass(
    out_path=workspace,
    out_name="Central_Points.shp",
    geometry_type="POINT",
    spatial_reference=spatial_ref
)

# Agregar campos a copiar
for fld in fields_to_copy:
    arcpy.management.AddField(central_points_fc, fld, "TEXT", field_length=50)

# Agregar campos X y Y
arcpy.management.AddField(central_points_fc, "X", "DOUBLE")
arcpy.management.AddField(central_points_fc, "Y", "DOUBLE")

# -----------------------------
# Insertar puntos centrales con coordenadas
# -----------------------------
with arcpy.da.SearchCursor(um_fc, ["SHAPE@"] + fields_to_copy) as search_cursor, \
     arcpy.da.InsertCursor(central_points_fc, ["SHAPE@", "X", "Y"] + fields_to_copy) as insert_cursor:
    
    for row in search_cursor:
        polygon = row[0]
        centroid = polygon.centroid  # esto es un Point
        x_coord = centroid.X        # corregido
        y_coord = centroid.Y        # corregido
        values = row[1:]            # UM_Code, LULC_Dom
        insert_cursor.insertRow([centroid, x_coord, y_coord] + list(values))

print(f"\n✅ Puntos centrales con coordenadas creados: {central_points_fc}")
print("🎉 Paso 13 completado correctamente")

In [ ]:
# ------------------------------------------------------------
# PASO 14 — GENERAR 3 PUNTOS POR CADA UM (Central, Norte, Sur)
# ------------------------------------------------------------

import arcpy
import os

# -----------------------------
# Rutas
# -----------------------------
workspace = r"C:\........"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

central_points = os.path.join(workspace, "Central_Points.shp")   # Shape de puntos centrales
out_points = os.path.join(workspace, "Central_Points_3perUM.shp")  # Nuevo shapefile con 3 puntos por UM

# -----------------------------
# Campos necesarios
# -----------------------------
um_field = "UM_Code"
lulc_field = "Strat_Dom"
x_field = "X"
y_field = "Y"

# -----------------------------
# Crear nuevo shapefile de salida
# -----------------------------
if arcpy.Exists(out_points):
    arcpy.Delete_management(out_points)

spatial_ref = arcpy.Describe(central_points).spatialReference

arcpy.CreateFeatureclass_management(
    out_path=workspace,
    out_name="Central_Points_3perUM.shp",
    geometry_type="POINT",
    spatial_reference=spatial_ref
)

# Agregar campos
arcpy.AddField_management(out_points, x_field, "DOUBLE")
arcpy.AddField_management(out_points, y_field, "DOUBLE")
arcpy.AddField_management(out_points, um_field, "TEXT", field_length=20)
arcpy.AddField_management(out_points, lulc_field, "TEXT", field_length=50)

# -----------------------------
# Crear cursores para insertar nuevos puntos
# -----------------------------
insert_fields = ["SHAPE@", x_field, y_field, um_field, lulc_field]

with arcpy.da.InsertCursor(out_points, insert_fields) as icursor:
    with arcpy.da.SearchCursor(central_points, ["SHAPE@", x_field, y_field, um_field, lulc_field]) as scursor:
        for row in scursor:
            x = row[1]
            y = row[2]
            um = row[3]
            lulc = row[4]

            # 1️⃣ Punto central
            icursor.insertRow((row[0], x, y, um, lulc))

            # 2️⃣ Punto al norte (Y + 80)
            icursor.insertRow((arcpy.Point(x, y + 80), x, y + 80, um, lulc))

            # 3️⃣ Punto al sur (Y - 80)
            icursor.insertRow((arcpy.Point(x, y - 80), x, y - 80, um, lulc))

print("\n🎉 Paso 14 completado: generado shapefile con 3 puntos por UM")
print(f"📂 Salida: {out_points}")

In [ ]:
# ------------------------------------------------------------
# PASO 15 — CREAR BUFFER DE 15 m (Parcelas circulares)
# ------------------------------------------------------------

import arcpy
import os

# -----------------------------
# Rutas
# -----------------------------
workspace = r"C:\........"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

points_fc = os.path.join(workspace, "Central_Points_3perUM.shp")  # Puntos de referencia
buffer_fc = os.path.join(workspace, "UM_Circular_Parcels.shp")     # Salida: parcelas circulares

# -----------------------------
# Crear buffer
# -----------------------------
if arcpy.Exists(buffer_fc):
    arcpy.Delete_management(buffer_fc)

# Buffer de 15 m de radio
arcpy.analysis.Buffer(
    in_features=points_fc,
    out_feature_class=buffer_fc,
    buffer_distance_or_field="15 Meters",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR"
)

print("\n🎉 Paso 15 completado: buffer de 15 m creado alrededor de cada punto")
print(f"📂 Salida: {buffer_fc}")